In [419]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [420]:
def linear_model(t_u, *params):
    result = 0
    for i, p in enumerate(params[:-1]):
      result += t_u[i] * p

    return result + params[-1:][0]

def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

def training_loop(n_epochs, model, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
  final_train_loss = -1
  final_val_loss = -1

  for epoch in range(1, n_epochs + 1):
    train_t_p = model(train_t_u, *params)
    train_loss = loss_fn(train_t_p, train_t_c)

    with torch.no_grad():
      val_t_p = model(val_t_u, *params)
      val_loss = loss_fn(val_t_p, val_t_c)
      assert val_loss.requires_grad == False

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch % 500 == 0 or epoch <= 3:
      print(f'Epoch={epoch}, training loss={train_loss.item():.4f}, validation loss={val_loss.item():.4f}')

    if epoch == n_epochs:
      final_train_loss = train_loss
      final_val_loss = val_loss

  return params, final_train_loss, final_val_loss

In [421]:
url = 'https://raw.githubusercontent.com/HamedTabkhi/Intro-to-ML/refs/heads/main/Dataset/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(url))

x = dataset[['area','bedrooms','bathrooms','stories','parking']]
y = dataset['price']

# scaler = StandardScaler()
# x = pd.DataFrame(scaler.fit_transform(x))

np.random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size = 0.2, random_state = 100)

train_t_u = [torch.tensor(x_train[col].values) for col in x_train.columns]
val_t_u = [torch.tensor(x_test[col].values) for col in x_test.columns]
train_t_c = torch.tensor(y_train.tolist())
val_t_c = torch.tensor(y_test.tolist())

In [422]:
learning_rate = [1e0, 1e-1, 1e-2, 1e-3, 1e-4]
final_train_loss = []
final_val_loss = []

for lr in learning_rate:
  print(f'Learning rate: {lr}')
  model = linear_model
  params = torch.tensor(np.ones((len(train_t_u)+1,1)),dtype=torch.float32, requires_grad=True)
  optimizer = optim.Adam([params], lr=lr)
  params, train_loss, val_loss = training_loop(5000, model, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c)
  final_train_loss.append(train_loss)
  final_val_loss.append(val_loss)
  print()


Learning rate: 1.0
Epoch=1, training loss=26423957913600.0000, validation loss=25105872715776.0000
Epoch=2, training loss=26370375680000.0000, validation loss=25053043359744.0000
Epoch=3, training loss=26316858458112.0000, validation loss=25000283209728.0000
Epoch=500, training loss=9190303072256.0000, validation loss=8320677576704.0000
Epoch=1000, training loss=4213831106560.0000, validation loss=3753279750144.0000
Epoch=1500, training loss=3426807971840.0000, validation loss=3186221383680.0000
Epoch=2000, training loss=3372349390848.0000, validation loss=3199549046784.0000
Epoch=2500, training loss=3369634627584.0000, validation loss=3208144224256.0000
Epoch=3000, training loss=3367770783744.0000, validation loss=3207288586240.0000
Epoch=3500, training loss=3365528666112.0000, validation loss=3205191172096.0000
Epoch=4000, training loss=3362905915392.0000, validation loss=3202698182656.0000
Epoch=4500, training loss=3359935299584.0000, validation loss=3199867551744.0000
Epoch=5000, t

In [423]:
print("Final losses:")
for i in range(0, len(learning_rate)):
  print(f'Learning rate={learning_rate[i]:.2e}, training loss={final_train_loss[i]:.2e}, validation loss={final_val_loss[i]:.2e}')

Final losses:
Learning rate=1.00e+00, training loss=3.36e+12, validation loss=3.20e+12
Learning rate=1.00e-01, training loss=8.60e+12, validation loss=7.76e+12
Learning rate=1.00e-02, training loss=2.38e+13, validation loss=2.26e+13
Learning rate=1.00e-03, training loss=2.62e+13, validation loss=2.48e+13
Learning rate=1.00e-04, training loss=2.64e+13, validation loss=2.51e+13
